In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import Select
import pandas as pd

## Let's do it for Andaman & Nicobar Island First

In [22]:
# Open a window of chrome using 
s=Service('C:/Users/sharm/Downloads/chromedriver-win32/chromedriver-win32/chromedriver.exe')
driver=webdriver.Chrome(service=s)

#Launching the website
driver.get("https://results.eci.gov.in/PcResultGenJune2024/index.htm")
# Give the page some time to load
time.sleep(1)

# Selecting the Option from dropdown
dropdown = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_Result1_ddlState')
select = Select(dropdown)
select.select_by_visible_text('Andaman & Nicobar Islands') 

#Finding the table path where the data is stored
table = driver.find_element(By.XPATH, '/html/body/main/div[2]/section/div/div/div[1]/div/div[2]/div/div/table') 

# Extract the data into a DataFrame
rows = table.find_elements(By.TAG_NAME, 'tr')
data = []
for row in rows:
    cols = row.find_elements(By.TAG_NAME, 'td')
    if cols:  # This skips the header row if present
        data.append([col.text for col in cols])
        
# Convert the list of data into a DataFrame
df = pd.DataFrame(data, columns=['Party', 'Won', 'Leading', 'Total'])

# Add a new column for the state
df['State'] = 'Andaman & Nicobar Islands'

driver.quit()

In [23]:
df

,Party,Won,Leading,Total,State
0,Bharatiya Janata Party - BJP,1,0,1,Andaman & Nicobar Islands


## Done for Andaman and Nicobar .... Now let's do it for all the states....

#### Using Beautiful Soup to Extract all statenames

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
url='https://results.eci.gov.in/PcResultGenJune2024/index.htm'
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}

In [34]:
webpage=requests.get(url=url, headers=headers).text

In [35]:
soup=BeautifulSoup(webpage, 'lxml')

In [39]:
dropdown=soup.find('select', attrs={'id': 'ctl00_ContentPlaceHolder1_Result1_ddlState'})

In [40]:
states = [option.text for option in dropdown.find_all('option') if option.text.strip() != "Select State Wise"]

In [41]:
states

['Andaman & Nicobar Islands',
 'Andhra Pradesh',
 'Arunachal Pradesh',
 'Assam',
 'Bihar',
 'Chandigarh',
 'Chhattisgarh',
 'Dadra & Nagar Haveli and Daman & Diu',
 'Goa',
 'Gujarat',
 'Haryana',
 'Himachal Pradesh',
 'Jammu and Kashmir',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Ladakh',
 'Lakshadweep',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Nagaland',
 'NCT OF Delhi',
 'Odisha',
 'Puducherry',
 'Punjab',
 'Rajasthan',
 'Sikkim',
 'Tamil Nadu',
 'Telangana',
 'Tripura',
 'Uttar Pradesh',
 'Uttarakhand',
 'West Bengal']

In [48]:
# Prepare a list to collect all data
all_data = []

for i in states:
    # Open a window of chrome using 
    s=Service('C:/Users/sharm/Downloads/chromedriver-win32/chromedriver-win32/chromedriver.exe')
    driver=webdriver.Chrome(service=s)

    #Launching the website
    driver.get("https://results.eci.gov.in/PcResultGenJune2024/index.htm")
    # Give the page some time to load
    time.sleep(1)
    
    # Selecting the Option from dropdown
    dropdown = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_Result1_ddlState')
    select = Select(dropdown)
    state_name=str(i)
    select.select_by_visible_text(state_name)

    # Extract table data
    table = driver.find_element(By.XPATH, '/html/body/main/div[2]/section/div/div/div[1]/div/div[2]/div/div/table')
    rows = table.find_elements(By.TAG_NAME, 'tr')
    for row in rows[1:]:  # Assuming the first row is headers
        cols = row.find_elements(By.TAG_NAME, 'td')
        if cols:
            row_data = [col.text for col in cols]
            all_data.append([state_name] + row_data)

    
# Convert all collected data into a DataFrame
columns = ['State', 'Party', 'Won', 'Leading', 'Total']  # Adjust column headers based on actual data
df_whole = pd.DataFrame(all_data, columns=columns)

# Clean up
driver.quit()


In [61]:
df_whole.head()

,State,Party,Won,Leading,Total
0,Andaman & Nicobar Islands,Bharatiya Janata Party - BJP,1,0,1
1,Andhra Pradesh,Telugu Desam - TDP,16,0,16
2,Andhra Pradesh,Yuvajana Sramika Rythu Congress Party - YSRCP,4,0,4
3,Andhra Pradesh,Bharatiya Janata Party - BJP,3,0,3
4,Andhra Pradesh,Janasena Party - JnP,2,0,2


### Data Cleaning

In [60]:
df_whole=df_whole.astype({'Won':int, 'Leading': int, 'Total':int})

In [62]:
df_whole['Total'].sum()

543

In [63]:
df_whole['Party'].value_counts()

Party
Bharatiya Janata Party - BJP                                             25
Indian National Congress - INC                                           23
Independent - IND                                                         6
Communist Party of India (Marxist) - CPI(M)                               3
Indian Union Muslim League - IUML                                         2
Voice of the People Party - VOTPP                                         1
Zoram People’s Movement - ZPM                                             1
Aam Aadmi Party - AAAP                                                    1
Shiromani Akali Dal - SAD                                                 1
Rashtriya Loktantrik Party - RLTP                                         1
Bharat Adivasi Party - BHRTADVSIP                                         1
Sikkim Krantikari Morcha - SKM                                            1
Dravida Munnetra Kazhagam - DMK                                           1
Shiv S

In [68]:
df_whole.Leading.value_counts()

Leading
0    96
Name: count, dtype: int64

In [70]:
df_whole.Won.sum()

543

In [72]:
df_whole.drop(columns=['Leading', 'Total'], inplace=True)

In [73]:
df_whole

,State,Party,Won
0,Andaman & Nicobar Islands,Bharatiya Janata Party - BJP,1
1,Andhra Pradesh,Telugu Desam - TDP,16
2,Andhra Pradesh,Yuvajana Sramika Rythu Congress Party - YSRCP,4
3,Andhra Pradesh,Bharatiya Janata Party - BJP,3
4,Andhra Pradesh,Janasena Party - JnP,2
...,...,...,...
91,Uttar Pradesh,Apna Dal (Soneylal) - ADAL,1
92,Uttarakhand,Bharatiya Janata Party - BJP,5
93,West Bengal,All India Trinamool Congress - AITC,29
94,West Bengal,Bharatiya Janata Party - BJP,12


In [86]:
df_whole['Party']=df_whole['Party'].apply(lambda x: str(x).split('-')[-1].strip())

In [88]:
df_whole.to_csv('Statewise_Partywise_Election_Results.csv', index=False)